In [1]:
import os
import sys
import numpy as np
import sklearn as skl
import tqdm
import pandas as pd
import time
import requests, zipfile, io

from data_utils.read_csv import *

## Download/Extract OpenEI Dataset

https://openei.org/datasets/files/961/pub/

In [5]:
dataset = 'https://openei.org/datasets/files/961/pub/EPLUS_TMY2_RESIDENTIAL_BASE.zip'
pathdir = './raw/'

if not os.path.exists(pathdir):
    r = requests.get(dataset)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall(pathdir)

In [21]:
pathdir = './raw/EPLUS_TMY2_RESIDENTIAL_BASE/'
files = sorted([f for f in os.listdir(pathdir) if os.path.isfile(os.path.join(pathdir, f))])
nfiles = len(files)
chosenfiles = 10
startidx = 5 # capture Big Delta, AK

In [22]:
load_features = ['DT', 'ELEC']
options = {}
Xs, Ys, filename = [], [], []
loadnum = 0
with tqdm.tqdm(total=chosenfiles) as pbar:
    for i,file in enumerate(files):
        if (i-startidx) % (nfiles//chosenfiles) is not 0:
            continue
        path = os.path.join(pathdir, file)
        print(path)
        df = read_load_csv(path,load_features)
        data, labels = df_to_numpy(df)
        x, y = encode_features(data, options)
        Xs.append(x)
        Ys.append(y)
        filename.append(file)
        loadnum += 1
        pbar.update(1)
X = np.vstack(Xs)
Y = np.vstack(Ys)

  0%|          | 0/10 [00:00<?, ?it/s]

./raw/EPLUS_TMY2_RESIDENTIAL_BASE/USA_AK_Big.Delta.702670_TMY2.csv


 10%|█         | 1/10 [00:01<00:10,  1.21s/it]

./raw/EPLUS_TMY2_RESIDENTIAL_BASE/USA_CA_Bakersfield.723840_TMY2.csv


 20%|██        | 2/10 [00:02<00:09,  1.21s/it]

...

 90%|█████████ | 9/10 [00:10<00:01,  1.21s/it]

./raw/EPLUS_TMY2_RESIDENTIAL_BASE/USA_UT_Salt.Lake.City.725720_TMY2.csv


100%|██████████| 10/10 [00:12<00:00,  1.22s/it]

./raw/EPLUS_TMY2_RESIDENTIAL_BASE/USA_WY_Rock.Springs.725744_TMY2.csv


11it [00:13,  1.22s/it]                        


## Save X and Y as torch tensors

In [23]:
import torch

X_torch = torch.tensor(X)
Y_torch = torch.tensor(Y)

output_dir = 'processed'
postfix = '_openei_%03d_subset_multitask.pt' %(loadnum)
if not os.path.exists(output_dir):
        os.mkdir(output_dir)   
        
torch.save(X_torch, os.path.join(output_dir, 'X'+postfix))
torch.save(Y_torch, os.path.join(output_dir, 'Y'+postfix))

## Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.1,random_state=1)
X_train_red, X_cv, Y_train_red, Y_cv = train_test_split(X_train, Y_train, train_size = 0.7/0.9,random_state=1)

## Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
models, r2, mse = [], [], []
for col in range(Y.shape[1]):
    reg = LinearRegression().fit(X_train, Y_train[:,col])
    models.append(reg)
    r2.append(reg.score(X_test,Y_test[:,col]))
    mse.append(mean_squared_error(Y_test[:,col],reg.predict(X_test)))
mse

## Cross-validated Lasso Regression

In [ ]:
from sklearn.linear_model import LassoCV
from sklearn.metrics import mean_squared_error
models, r2, mse = [], [], []
for col in range(Y.shape[1]):
    reg = LassoCV(cv=5,random_state=0).fit(X_train, Y_train[:,col])
    models.append(reg)
    r2.append(reg.score(X_test,Y_test[:,col]))
    mse.append(mean_squared_error(Y_test[:,col],reg.predict(X_test)))
mse

## Decision Tree Regression

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
models, r2, mse = [], [], []
min_depth, max_depth = 2,15

with tqdm.tqdm(total=Y.shape[1]*(max_depth+1-min_depth)) as pbar:
    for col in range(Y.shape[1]):

        # hyperparameter search over depth
        model_options, cv_mse = [], []
        for d in range(min_depth,max_depth+1):
            reg = DecisionTreeRegressor(max_depth=d).fit(X_train_red, Y_train_red[:,col])
            model_options.append(reg)
            cv_mse.append(mean_squared_error(Y_cv[:,col],reg.predict(X_cv)))
            pbar.update(1)
            
        best_cv_idx = np.array(cv_mse).argmax()
        print("Best tree depth for column %d: %d" %(col,best_cv_idx+3))
        reg = model_options[best_cv_idx]
        models.append(reg)
        r2.append(reg.score(X_test,Y_test[:,col]))
        mse.append(mean_squared_error(Y_test[:,col],reg.predict(X_test)))

mse

## Support Vector Regression

In [ ]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
import itertools

models, r2, mse = [], [], []
#hyperparameters
kernels = ['poly','rbf']
Cs = [0.2,1.0,5.0] #[0.1,0.2,0.5,1.0,2.0,5.0,10.0]

with tqdm.tqdm(total=Y.shape[1]*len(kernels)*len(Cs)) as pbar:
    for col in range(Y.shape[1]):

        # hyperparameter search
        model_options, cv_mse = [], []
        for kernel, C in list(itertools.product(kernels, Cs)):
            reg = SVR(kernel=kernel, C=C).fit(X_train_red, Y_train_red[:,col])
            model_options.append(reg)
            cv_mse.append(mean_squared_error(Y_cv[:,col],reg.predict(X_cv)))
            pbar.update(1)
            
        best_cv_idx = np.array(cv_mse).argmax()
        print("Best hyperparameter options for column %d: %s" %(col,list(itertools.product(kernels, Cs))[best_cv_idx]))
        reg = model_options[best_cv_idx]
        models.append(reg)
        r2.append(reg.score(X_test,Y_test[:,col]))
        mse.append(mean_squared_error(Y_test[:,col],reg.predict(X_test)))

mse

In [ ]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

models, r2, mse = [], [], []
#hyperparameters
parameters = [
  {'C': [1, 10, 100, 1000], 'kernel': ['poly']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.1, 0.01, 0.001], 'kernel': ['rbf']},
 ]
parameters = [
  {'C': [1, 10], 'gamma': [0.1, 0.01], 'kernel': ['rbf']},
 ]
with tqdm.tqdm(total=Y.shape[1]) as pbar:
    for col in range(Y.shape[1]):

        # hyperparameter search
        reg = SVR()
        clf = GridSearchCV(reg, parameters, scoring='neg_mean_squared_error', cv=5, n_jobs=-1)
        clf.fit(X_train, Y_train[:,col])
        
        pbar.update(1)
        
        models.append(clf)
        r2.append(clf.score(X_test,Y_test[:,col]))
        mse.append(mean_squared_error(Y_test[:,col],clf.predict(X_test)))

mse

## Multiple Shallow Neural Nets

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

parameters = {'hidden_layer_sizes': [(10,10),(20,20),(30,30),(20,20,20),(30,30,30)]}
models, r2, mse = [], [], []
with tqdm.tqdm(total=Y.shape[1]) as pbar:
    for col in range(Y.shape[1]):
        mlp = MLPRegressor(learning_rate='adaptive')
        reg = GridSearchCV(mlp, parameters, scoring='neg_mean_squared_error', cv=5, n_jobs=-1)
        reg.fit(X_train, Y_train[:,col])
        models.append(reg)
        r2.append(reg.score(X_test,Y_test[:,col]))
        mse.append(mean_squared_error(Y_test[:,col],reg.predict(X_test)))
        pbar.update(1)
mse

## Neural Net w/ Shared Weights

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

models, r2, mse = [], [], []
parameters = {'hidden_layer_sizes': [(10,10),(20,20),(30,30),(50,50),(100,100),(20,20,20),(30,30,30),
                                     (50,50,50),(20,20,20,20),(30,30,30,30)]}
#parameters = {'hidden_layer_sizes': [(30,30,30)]}
mlp = MLPRegressor(learning_rate='adaptive')
reg = GridSearchCV(mlp,parameters, scoring='neg_mean_squared_error', cv=5, n_jobs=-1)
reg.fit(X_train, Y_train)
models.append(reg)
r2.append(reg.score(X_test,Y_test))

Y_test_pred = reg.predict(X_test)
for col in range(Y_test.shape[1]):
    mse.append(mean_squared_error(Y_test_pred[col], Y_test[col]))
mse

## Dirty MTL Model

$W = P+Q$, $W$ of size (input features, output tasks)

$Q$ is element-wise sparse and is regularized with $\lVert Q \rVert_1$

$P$ is low-rank and is regularized with $\lVert P \rVert_*$ or row-wise group-sparse, regularized with $\lVert P \rVert_{1,q}$

The p,q norm takes the q-norm of each row (single feature to multiple tasks), then the p-norm over q-norms.

The * norm is the sum of singular values obtained from SVD

## Robust MTL Model

$W = P+Q$, $W$ of size (input features, output tasks)

$Q$ is column-wise group sparse (against outlier tasks) and is regularized with $\lVert Q^T \rVert_{1,q}$

$P$ is low-rank and is regularized with $\lVert P \rVert_*$ or row-wise group-sparse, regularized with $\lVert P \rVert_{1,q}$